In [32]:
# Configuration Constants
import os
from openai import OpenAI, AzureOpenAI
from botocore.exceptions import ClientError
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Base URLs from environment
BASE_URL = os.getenv("GOOP_API_URL", "http://localhost:8080")
API_KEY = os.getenv("GOOP_API_KEY", "test")

OPENAI_ENDPOINT = f"{BASE_URL}/openai/v1"
OPENAI_PROXY_ENDPOINT = f"{BASE_URL}/openai-proxy/v1"
AZURE_ENDPOINT = f"{BASE_URL}/azure"
BEDROCK_ENDPOINT = f"{BASE_URL}/bedrock"
GEMINI_ENDPOINT = f"{BASE_URL}/gemini"

# Environment Variables
GEMINI_PROJECT_ID = "encrypted-llm"

# Models
OLLAMA_MODEL = "llama3.2:latest"
BEDROCK_SONNET_4 = "us.anthropic.claude-sonnet-4-20250514-v1:0"
GEMINI_FLASH_LITE = "gemini-2.5-flash-lite"
OPENAI_GPT4O_MINI = "gpt-4o-mini"

print(f"Configuration loaded successfully!")
print(f"Base URL: {BASE_URL}")
print(f"API Key: {'***' + API_KEY[-4:] if len(API_KEY) > 4 else '***'}")

Configuration loaded successfully!
Base URL: https://xypcggtuxrriwuoxdjkwfitfjq0wrzli.lambda-url.us-east-1.on.aws/
API Key: ***ea8d


# Base OpenAI Client

```python
from openai import OpenAI, AzureOpenAI
client = OpenAI(
    base_url="http://localhost:8080/openai/v1",
    api_key="test",
)
```

In [33]:
#!/usr/bin/env python3

client = OpenAI(
    base_url=OPENAI_ENDPOINT,
    api_key=API_KEY,
)

print(client.models.list())

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model=f"gpt-4o-mini",
    stream=False,
)

print(chat_completion.choices[0].message.content)

SyncPage[Model](data=[Model(id='gpt-4-0613', created=1686588896, object='model', owned_by='openai'), Model(id='gpt-4', created=1687882411, object='model', owned_by='openai'), Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'), Model(id='o4-mini-deep-research-2025-06-26', created=1750866121, object='model', owned_by='system'), Model(id='codex-mini-latest', created=1746673257, object='model', owned_by='system'), Model(id='gpt-4o-realtime-preview-2025-06-03', created=1748907838, object='model', owned_by='system'), Model(id='gpt-4o-audio-preview-2025-06-03', created=1748908498, object='model', owned_by='system'), Model(id='o4-mini-deep-research', created=1749685485, object='model', owned_by='system'), Model(id='davinci-002', created=1692634301, object='model', owned_by='system'), Model(id='babbage-002', created=1692634615, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'), Model(id='gp

In [13]:
### Streaming
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Write me a pretty short story",
        }
    ],
    model="gpt-4o-mini",
    stream=True,
)

for i in chat_completion:
    print(i.choices[0].delta.content, end="")
print()

Once in a quiet village nestled between rolling hills, there lived an old woman named Elara. She was known for her vibrant garden, a riot of colors that danced under the sun. Every spring, villagers would gather to admire her blooms, drawn by the sweet scent of blossoms that filled the air.

One day, a young girl named Lily approached Elara with wide eyes. "How do you make your flowers grow so beautiful?" she asked. 

Elara smiled, her eyes twinkling like the stars. "It’s simple, my dear. I talk to them."

Lily giggled. "You can't talk to flowers!"

Elara chuckled, then knelt beside a sunflower towering over them. "Watch closely." She softly whispered words of encouragement to the flower, and in response, it swayed gently, as if it understood.

Curious, Lily began visiting Elara daily. Together, they nurtured the garden, each conversation weaving a bond between them and the plants. As the days passed, Lily learned not just the art of gardening but the beauty of connection — with nature

In [34]:
### Streaming with OPenai Proxy
client = OpenAI(
    base_url=OPENAI_PROXY_ENDPOINT,
    api_key=API_KEY,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Write me a pretty short story",
        }
    ],
    model=f"gemini/{GEMINI_FLASH_LITE}",
    stream=True,
)

for i in chat_completion:
    print(i.choices[0].delta.content, end="")
print()

The old woman, Elara, sat by the window, sunlight warming her face. Her fingers, gnarled like ancient roots, traced the patterns on a worn quilt. Outside, a single red robin hopped across the dew-kissed grass, its song a delicate thread woven into the morning air.

For years, this small cottage had been her world. The scent of drying herbs and the whisper of the wind through the oak tree were her companions. Today, however, a different kind of whisper stirred. It was a memory, sharp and clear as the robin's eye, of a day long ago when she'd planted a tiny sapling in this very spot.

She’d been young then, her hands smooth and her heart full of a hopeful impatience. The sapling had seemed so fragile, so insignificant. Now, its broad branches reached towards the sky, a testament to time and quiet resilience.

The robin chirped again, a bright punctuation mark in her thoughts. Elara smiled, a soft, knowing curve of her lips. Life, like the robin, found its way, even in the stillness. And 

### Boto3 Bedrock Runtime Client

```python
def _replace_headers(request: AWSRequest, **kwargs):
    request.headers = {"Authorization": "Bearer test"}

client = boto3.client("bedrock-runtime", endpoint_url="http://localhost:8080/bedrock")
client.meta.events.register("before-send.bedrock-runtime.*", _replace_headers)
```

In [16]:
import boto3
from botocore.exceptions import ClientError


def _replace_headers(request, **kwargs):
    request.headers = {"Authorization": "Bearer " + API_KEY}


bedrockClient = boto3.client(
    "bedrock-runtime", endpoint_url=f"{BASE_URL}/bedrock", region_name="us-east-1"
)
bedrockClient.meta.events.register("before-send.bedrock-runtime.*", _replace_headers)

model_id = BEDROCK_SONNET_4  # "us.anthropic.claude-sonnet-4-20250514-v1:0"
user_message = "Say 'hi'"
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    response = bedrockClient.converse(
        modelId=model_id,
        messages=conversation,
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
    )
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

Hi!


### Google Vertex Client

```python
import vertexai
from vertexai.preview.generative_models import GenerativeModel

vertexai.init(
    project=os.getenv("VERTEX_AI_PROJECT_ID"),
    api_endpoint="http://localhost:8080/vertex",
)

generative_multimodal_model = GenerativeModel("gemini-1.5-flash-002")
response = generative_multimodal_model.generate_content(["Say hi"])

print(response)
```

In [ ]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
import os
from google import genai
from google.genai import types


genai_client = genai.Client(
    http_options=types.HttpOptions(
        base_url=GEMINI_ENDPOINT, headers={"Authorization": f"Bearer {API_KEY}"}
    ),
    api_key=API_KEY,
)

model = "gemini-2.5-flash-lite"
contents = [
    types.Content(
        role="user",
        parts=[
            types.Part.from_text(text="""INSERT_INPUT_HERE"""),
        ],
    ),
]
tools = [
    types.Tool(googleSearch=types.GoogleSearch()),
]
generate_content_config = types.GenerateContentConfig(
    thinking_config=types.ThinkingConfig(
        thinking_budget=0,
    ),
    tools=tools,
)

for chunk in genai_client.models.generate_content_stream(
    model=model,
    contents=contents,
    config=generate_content_config,
):
    print(chunk.text, end="")

ClientError: 404 Not Found. {'message': 'Engine not found\n', 'status': 'Not Found'}

### ELL Framework Example

In [50]:
import ell
from pydantic import Field
import requests
from bs4 import BeautifulSoup

openai_client = OpenAI(
    base_url=OPENAI_PROXY_ENDPOINT,
    api_key=API_KEY,
)

ell.init(verbose=True)


"""
BEDROCK CLIENT
"""


@ell.simple(model=f"gemini/{GEMINI_FLASH_LITE}", client=openai_client)
def bedrock_chat(prompt: str) -> str:
    return prompt


print(f'Bedrock \n\n {bedrock_chat("Hello, how are you?")}\n\n')

╔══════════════════════════════════════════════════════════════════════════════╗
║ bedrock_chat(Hello, h..)
╠══════════════════════════════════════════════════════════════════════════════╣
║ Prompt:
╟──────────────────────────────────────────────────────────────────────────────╢
│        user: Hello, how are you?
╟──────────────────────────────────────────────────────────────────────────────╢
║ Output:
╟──────────────────────────────────────────────────────────────────────────────╢
│   assistant: I am doing well,
│              thank you for asking! I am a large language model, trained by Google. How
│              can I help you today?
╚══════════════════════════════════════════════════════════════════════════════╝
Bedrock 

 I am doing well, thank you for asking! I am a large language model, trained by Google. How can I help you today?




In [38]:
"""
OpenAI Client
"""

@ell.simple(model="gpt-4o-mini", max_tokens=10, client=client)
def openai_client(prompt: str) -> str:
    return prompt


print(f'OpenAI \n\n {openai_client("Hello, how are you?")}\n\n')

╔══════════════════════════════════════════════════════════════════════════════╗
║ openai_client(Hello, h..)
╠══════════════════════════════════════════════════════════════════════════════╣
║ Prompt:
╟──────────────────────────────────────────────────────────────────────────────╢
│        user: Hello, how are you?
╟──────────────────────────────────────────────────────────────────────────────╢
║ Output:
╟──────────────────────────────────────────────────────────────────────────────╢
│   assistant: Hello! I'm just a program, but I'm here
╚══════════════════════════════════════════════════════════════════════════════╝
OpenAI 

 Hello! I'm just a program, but I'm here




In [40]:
"""
TOOL USAGE
"""

@ell.tool()
def get_html_content(
    url: str = Field(
        description="The URL to get the HTML content of. Never include the protocol (like http:// or https://)"
    ),
):
    """Get the HTML content of a URL."""
    response = requests.get("https://" + url)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup.get_text()[:100]


# OpenAI Client
@ell.complex(
    model="gpt-4o-mini",
    tools=[get_html_content],
    client=client,
)
def openai_get_website_content(website: str) -> str:
    return f"Tell me what's on {website}"


print("OpenAI Client Tool Use\n\n")
output = openai_get_website_content("new york times front page")
if output.tool_calls:
    print(output.tool_calls[0]())


# Bedrock Client
@ell.complex(
    model="anthropic.claude-3-haiku-20240307-v1:0",
    tools=[get_html_content],
    client=bedrockClient,
)
def bedrock_get_website_content(website: str) -> str:
    """You are an agent that can summarize the contents of a website."""
    return f"Tell me what's on {website}"


print("\n\nBedrock Client Tool Use\n\n")
output = bedrock_get_website_content("new york times front page")
if output.tool_calls:
    print(output.tool_calls[0]())

OpenAI Client Tool Use


╔══════════════════════════════════════════════════════════════════════════════╗
║ openai_get_website_content(new york..)
╠══════════════════════════════════════════════════════════════════════════════╣
║ Prompt:
╟──────────────────────────────────────────────────────────────────────────────╢
│        user: Tell me what's on new york times front page
╟──────────────────────────────────────────────────────────────────────────────╢
║ Output:
╟──────────────────────────────────────────────────────────────────────────────╢
│   assistant: 
│              ChatCompletionMessageToolCall(id='call_dhg9F4eXIY8Evev2FT0EvjO8', function=Function(arguments='{"url":"nytimes.com"}', name='get_html_content'), type='function')
╚══════════════════════════════════════════════════════════════════════════════╝




The New York Times - Breaking News, US News, World News and Videos
  





























Bedrock Client Tool Use


╔══════════════════════════════════════════════

In [42]:
from openai import OpenAI, AzureOpenAI

client = OpenAI(
    base_url=OPENAI_PROXY_ENDPOINT,
    api_key=API_KEY,
)
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Whats up dog?",
        }
    ],
    model="bedrock/us.anthropic.claude-3-haiku-20240307-v1:0",
    stream=False,
)

print(chat_completion)
print(chat_completion.choices[0].message.content)

ChatCompletion(id='chatcmpl-20250804050300', choices=[Choice(finish_reason='end_turn', index=0, logprobs=None, message=ChatCompletionMessage(content='I\'m an AI assistant. I don\'t actually have a physical form or the ability to be a "dog". I\'m happy to chat with you, but I don\'t have the same kind of casual, conversational abilities that a human would.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1754283780, model='bedrock-claude', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=53, prompt_tokens=18, total_tokens=71, completion_tokens_details=None, prompt_tokens_details=None))
I'm an AI assistant. I don't actually have a physical form or the ability to be a "dog". I'm happy to chat with you, but I don't have the same kind of casual, conversational abilities that a human would.


In [43]:
from openai import OpenAI, AzureOpenAI

client = OpenAI(
    base_url=OPENAI_PROXY_ENDPOINT,
    api_key=API_KEY,
)
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Write me a pretty short story",
        }
    ],
    model="bedrock/us.anthropic.claude-3-sonnet-20240307-v2:0",
    stream=True,
)

# for i in chat_completion:
#     print(i)
# print()


for i in chat_completion:
    print(i.choices[0].delta.content, end="")
print()

In [45]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID.",
                    },
                },
                "required": ["order_id"],
                "additionalProperties": False,
            },
        },
    }
]

messages = [
    {"role": "user", "content": "Hi, can you tell me the delivery date for my order? Its order id 12345."},
]


chat_completion = client.chat.completions.create(
    messages=messages,
    model=f"bedrock/{BEDROCK_SONNET_4}",
    stream=False,
    tools=tools,
    tool_choice="required",
)
print(chat_completion.choices[0])

Choice(finish_reason='tool_use', index=0, logprobs=None, message=ChatCompletionMessage(content="I'll help you check the delivery date for your order. Let me look that up for you.", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='tooluse_26hyeCe9RESYL5K6OOzjxA', function=Function(arguments={'order_id': '12345'}, name='get_delivery_date'), type='function')]))


In [ ]:
print(client.models.list())